In [90]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import datetime

data = pd.read_csv('datasets_8127_11403_Market_Basket_Optimisation.csv',header=None)
data1 = data.copy()
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
#apriori算法
data['ColumnA'] = data[data.columns[0:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)
data_merg =  pd.DataFrame(data=data['ColumnA'].values, columns=['ColumnA'])
data_hot_encoded = data_merg.drop('ColumnA', 1).join(data_merg['ColumnA'].str.get_dummies(','))
data_hot_encoded.head()
begin = datetime.datetime.now()
itemsets = apriori(data_hot_encoded, use_colnames=True, min_support=0.01)
end = datetime.datetime.now()
print("apriori cost time:%u" % (end-begin).seconds)
itemsets = itemsets.sort_values(by='support', ascending=False)
pd.options.display.max_columns = 100
rules = association_rules(itemsets, metric='lift', min_threshold=2)
rules = rules.sort_values(by='lift',ascending=False)
rules.head()



2020-07-25 14:26:41.082807
2020-07-25 14:26:41.551554
apriori cost time:0


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
10,(herb & pepper),(ground beef),0.049460,0.098254,0.015998,0.323450,3.291994,0.011138,1.332860
11,(ground beef),(herb & pepper),0.098254,0.049460,0.015998,0.162822,3.291994,0.011138,1.135410
2,"(spaghetti, mineral water)",(ground beef),0.059725,0.098254,0.017064,0.285714,2.907928,0.011196,1.262445
5,(ground beef),"(spaghetti, mineral water)",0.098254,0.059725,0.017064,0.173677,2.907928,0.011196,1.137902
44,"(spaghetti, mineral water)",(olive oil),0.059725,0.065858,0.010265,0.171875,2.609786,0.006332,1.128021


In [91]:
#fpgrowth 算法
import pyfpgrowth
import numpy as np
transactions= [list(i[~pd.isnull(i)]) for i in data1.values]
transactions[0:10]
begin = datetime.datetime.now()
patterns = pyfpgrowth.find_frequent_patterns(transactions, 100)
end = datetime.datetime.now()
print(begin)
print(end)
print("fpgrowth cost time:",format((end-begin).seconds))
rules = pyfpgrowth.generate_association_rules(patterns, 1)



2020-07-25 14:56:08.618936
2020-07-25 14:56:10.738245
fpgrowth cost time: 2


In [114]:
data.shape



(7501, 20)

In [174]:
%%time
itemsets = apriori(data_hot_encoded, use_colnames=True, min_support=300/data.shape[0])
print(len(itemsets))
print(sorted(itemsets['itemsets'].values))

35
[frozenset({'burgers'}), frozenset({'cake'}), frozenset({'champagne'}), frozenset({'chicken'}), frozenset({'chocolate'}), frozenset({'cookies'}), frozenset({'cooking oil'}), frozenset({'eggs'}), frozenset({'escalope'}), frozenset({'french fries'}), frozenset({'fresh bread'}), frozenset({'frozen smoothie'}), frozenset({'frozen vegetables'}), frozenset({'grated cheese'}), frozenset({'green tea'}), frozenset({'ground beef'}), frozenset({'herb & pepper'}), frozenset({'honey'}), frozenset({'low fat yogurt'}), frozenset({'milk'}), frozenset({'mineral water'}), frozenset({'olive oil'}), frozenset({'pancakes'}), frozenset({'salmon'}), frozenset({'shrimp'}), frozenset({'soup'}), frozenset({'spaghetti'}), frozenset({'tomatoes'}), frozenset({'turkey'}), frozenset({'whole wheat rice'}), frozenset({'chocolate', 'mineral water'}), frozenset({'mineral water', 'eggs'}), frozenset({'mineral water', 'ground beef'}), frozenset({'milk', 'mineral water'}), frozenset({'spaghetti', 'mineral water'})]
Wall

In [179]:
%%time
patterns = pyfpgrowth.find_frequent_patterns(transactions, 200)
print(len(patterns))
print(sorted(patterns.keys()))

60
[('avocado',), ('brownies',), ('burgers',), ('burgers', 'eggs'), ('butter',), ('cake',), ('cake', 'mineral water'), ('champagne',), ('chicken',), ('chocolate', 'eggs'), ('chocolate', 'french fries'), ('chocolate', 'milk'), ('chocolate', 'mineral water'), ('chocolate', 'spaghetti'), ('cookies',), ('cooking oil',), ('cottage cheese',), ('eggs',), ('eggs', 'french fries'), ('eggs', 'milk'), ('eggs', 'mineral water'), ('eggs', 'spaghetti'), ('energy bar',), ('energy drink',), ('escalope',), ('french fries', 'green tea'), ('french fries', 'mineral water'), ('french fries', 'spaghetti'), ('fresh bread',), ('frozen smoothie',), ('frozen vegetables', 'mineral water'), ('frozen vegetables', 'spaghetti'), ('grated cheese',), ('green tea', 'mineral water'), ('ground beef', 'mineral water'), ('ground beef', 'spaghetti'), ('ham',), ('herb & pepper',), ('honey',), ('hot dogs',), ('light mayo',), ('low fat yogurt',), ('milk', 'mineral water'), ('milk', 'spaghetti'), ('mineral water',), ('mineral w

In [178]:
%%time
import fptools as fp
mfis = [iset for iset in fp.frequent_itemsets(transactions, 200)]
print(len(mfis))
print(sorted(mfis))


67
[['avocado'], ['brownies'], ['burgers'], ['burgers', 'eggs'], ['butter'], ['cake'], ['cake', 'mineral water'], ['champagne'], ['chicken'], ['chocolate'], ['chocolate', 'eggs'], ['chocolate', 'french fries'], ['chocolate', 'mineral water'], ['chocolate', 'spaghetti'], ['cookies'], ['cooking oil'], ['cottage cheese'], ['eggs'], ['eggs', 'mineral water'], ['energy bar'], ['energy drink'], ['escalope'], ['french fries'], ['french fries', 'eggs'], ['french fries', 'mineral water'], ['french fries', 'spaghetti'], ['fresh bread'], ['frozen smoothie'], ['frozen vegetables'], ['frozen vegetables', 'mineral water'], ['frozen vegetables', 'spaghetti'], ['grated cheese'], ['green tea'], ['green tea', 'french fries'], ['green tea', 'mineral water'], ['ground beef'], ['ground beef', 'mineral water'], ['ground beef', 'spaghetti'], ['ham'], ['herb & pepper'], ['honey'], ['hot dogs'], ['light mayo'], ['low fat yogurt'], ['milk'], ['milk', 'chocolate'], ['milk', 'eggs'], ['milk', 'mineral water'], ['

In [177]:
%%time
import fptools as fp
tree = fp.build_tree(transactions, minsup=300)
items =[i for i in fp.fpgrowth(tree[0], 50)]
print(len(items))

359
Wall time: 84.8 ms
